In [ ]:
# WARNING: this is only a prototype and still work in progress.

import base64
import io

import ipyreact
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from skimage import data
from skimage.color import label2rgb
from skimage.filters import sobel
from skimage.measure import label
from skimage.segmentation import watershed
from traitlets import Unicode, Int, observe

def generate_image_assets():
    coins = data.coins()
    edges = sobel(coins)
    markers = np.zeros(coins.shape, dtype=np.uint)

    fig, ax = plt.subplots()
    ax.axis("off")
    ax.imshow(coins, cmap="gray")
    plt.tight_layout()
    buf = io.BytesIO()
    fig.savefig(
        buf, format="png", bbox_inches="tight", transparent="True", pad_inches=0
    )
    plt.close()
    buf.seek(0)

    base64_img_string_pre = base64.b64encode(buf.getvalue()).decode()
    base64_img_string = f"data:image/png;base64,{base64_img_string_pre}"
    b64_string = base64_img_string_pre
    decoded_bytes = base64.b64decode(b64_string)

    ihdr_start = 8
    ihdr_end = decoded_bytes.find(b"IHDR") + 4 + 8
    ihdr_chunk = decoded_bytes[ihdr_start:ihdr_end]
    image_width = int.from_bytes(ihdr_chunk[8:12], byteorder="big")
    image_height = int.from_bytes(ihdr_chunk[12:16], byteorder="big")

    return base64_img_string, image_width, image_height, coins, edges, markers

# TODO: this can still be improved
base64_img_string, image_width, image_height, coins, edges, markers = generate_image_assets()

def segment_image_based_on_marker(marker_coord):
    coord = marker_coord.split(",")
    coord = [int(float(i)) for i in coord]
    coord[1] = coord[1] - 60  # because tldraw canvas is image_height+120
    coord = [coord[1], coord[0]]  # because of different coordinate system
    # because of image scaling in canvas, could be improved
    coord[0] = int(coord[0] / 1.43)
    coord[1] = int(coord[1] / 1.43)

    foreground, background = 1, 2
    markers[coins < 30.0] = background
    markers[coord[0], coord[1]] = foreground

    # based on https://scikit-image.org/docs/stable/auto_examples/segmentation/plot_expand_labels.html#sphx-glr-auto-examples-segmentation-plot-expand-labels-py
    ws = watershed(edges, markers)
    seg1 = label(ws == foreground)

    fig, ax = plt.subplots()
    color1 = label2rgb(seg1, image=coins, bg_label=0)
    ax.axis("off")
    ax.imshow(color1)

    buf = io.BytesIO()
    fig.savefig(
        buf, format="png", bbox_inches="tight", transparent="True", pad_inches=0
    )
    plt.close()
    buf.seek(0)
    base64_img_string_pre = base64.b64encode(buf.getvalue()).decode()
    base64_img_string = f"data:image/png;base64,{base64_img_string_pre}"

    return base64_img_string

class TldrawSegmentation(ipyreact.ReactWidget):
    my_coordinates = Unicode("Hello World").tag(sync=True)
    base64img = Unicode("").tag(sync=True)
    image_width = Int(300).tag(sync=True)
    image_height = Int(100).tag(sync=True)

    @observe("my_coordinates")
    def _observe_count(self, change):
        self.base64img = segment_image_based_on_marker(self.my_coordinates)

    _esm = """
    import { TDAssetType, TDShapeType, Tldraw, TldrawApp } from "@tldraw/tldraw";
    import * as React from "react";

    export default function App({
      image_width,
      image_height,
      base64img,
      set_my_coordinates,
    }) {
      const [app, setApp] = React.useState();

      const handleMount = React.useCallback((app: Tldraw) => {
        setApp(app);
      }, []);

      React.useEffect(() => {
        if (app) {
          app.patchAssets({
            myAssetId: {
              id: "myAssetId",
              type: TDAssetType.Image,
              size: [image_width, image_height],
              fileName: "card-repo.png",
              src: base64img,
            },
          });

          app.createShapes({
            id: "myImage",
            type: TDShapeType.Image,
            assetId: "myAssetId",
            point: [0, 60],
            size: [image_width, image_height],
            isLocked: true,
          });
        }
      }, [base64img, app]);

      const handleChange = (e) => {
        let my_current_stroke = e.selectedIds[0];
        let my_object = e.getShape(my_current_stroke);

        if (my_object === undefined) {
          console.log("no object selected");
        }
        if (my_object !== undefined) {
          let my_points = my_object.points;
          if (my_points !== undefined) {
            console.log(my_object.id);
            console.log(my_points.length);
            console.log(my_points[my_points.length - 1]);
            set_my_coordinates(my_object.point.toString());
          }
        }
      };

      return (
        <div
          style={{
            position: "relative",
            width: image_width,
            height: image_height + 120,
          }}
        >
          <Tldraw
            onMount={handleMount}
            onChange={handleChange}
            showMenu={false}
            showPages={false}
          />
        </div>
      );
    }
    """

tldraw_segmentation = TldrawSegmentation(image_width=image_width, image_height=image_height, base64img=base64_img_string)
# In the canvas select "Draw" to draw strokes on the canvas. The segmentation is updated automatically.
display(tldraw_segmentation)